In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import to_json,col
from pyspark.sql.types import *
from os.path import abspath

spark = SparkSession\
        .builder\
        .appName("pyspark-notebook")\
        .master("spark://spark-master:7077")\
        .config("spark.executor.memory", "512m")\
        .config("hive.metastore.uris", "thrift://hive-metastore:9083")\
        .config("spark.sql.warehouse.dir", "/user/hive/warehouse")\
        .enableHiveSupport()\
        .getOrCreate()

spark.sparkContext.setLogLevel("ERROR")

24/10/12 19:11:02 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


## The Data

In [3]:
!apt install wget -y

Reading package lists... Done
Building dependency tree       
Reading state information... Done
wget is already the newest version (1.20.1-1.1).
0 upgraded, 0 newly installed, 0 to remove and 0 not upgraded.


In [6]:
!wget 'https://drive.google.com/uc?export=download&id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u' -O uplift_data.zip

--2024-10-12 19:19:38--  https://drive.google.com/uc?export=download&id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u
Resolving drive.google.com (drive.google.com)... 209.85.233.194, 2a00:1450:4010:c03::c2
Connecting to drive.google.com (drive.google.com)|209.85.233.194|:443... connected.
HTTP request sent, awaiting response... 303 See Other
Location: https://drive.usercontent.google.com/download?id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u&export=download [following]
--2024-10-12 19:19:39--  https://drive.usercontent.google.com/download?id=1wdytBkoYRyaNNv3XUNnwcGbdNUSfok7u&export=download
Resolving drive.usercontent.google.com (drive.usercontent.google.com)... 142.250.74.65, 2a00:1450:400f:802::2001
Connecting to drive.usercontent.google.com (drive.usercontent.google.com)|142.250.74.65|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 8886971 (8.5M) [application/octet-stream]
Saving to: ‘uplift_data.zip’

uplift_data.zip     100%[===================>]   8.47M  2.63MB/s    in 3.2s 

In [7]:
!unzip uplift_data.zip

Archive:  uplift_data.zip
  inflating: clients.csv             
  inflating: uplift_test.csv         
  inflating: uplift_train.csv        


In [8]:
ls

None         pyspark.ipynb*   uplift_test.csv
clients.csv  uplift_data.zip  uplift_train.csv


In [12]:
data = spark.read.csv(path="clients.csv", sep=",", header=True)

In [13]:
data.count()

400162

In [14]:
data.head()

Row(client_id='000012768d', first_issue_date='2017-08-05 15:40:48', first_redeem_date='2018-01-04 19:30:07', age='45', gender='U')

In [15]:
df1 = data.select(["client_id", "age", "gender"])

In [16]:
df1.head()

Row(client_id='000012768d', age='45', gender='U')

In [18]:
df1 = data.select(['client_id', 'age'])

In [20]:
cols_description = df1.filter(df1['age'] == '45')

In [21]:
df1 = df1.join(other=cols_description, on=['age'], how='left_anti')

In [22]:
df1 = df1.withColumnRenamed('client_id', 'c1').withColumnRenamed('age', 'c2').withColumnRenamed('gender', 'c3')

In [27]:
df1.write.saveAsTable('hive_table')

AnalysisException: Table `hive_table` already exists.;

In [24]:
ndf = spark.sql('''
select * from hive_table
''')

In [26]:
ndf.show(10)

+---+----------+
| c2|        c1|
+---+----------+
| 72|000036f903|
| 68|000048b7a6|
| 60|000073194a|
| 67|00007c7133|
| 46|0000b59cec|
| 36|0000bb4e4e|
| 34|0000bcec9c|
| 70|0000eecb82|
| 56|0000f0ecdb|
| 83|00010925a5|
+---+----------+
only showing top 10 rows

